***THIS IS A PROJECT ON EXPLORATORY DATA ANALYSIS***

In [101]:
import numpy as np
import pandas as pd

# 1) READING DATA USING PANDAS

In [102]:
df=pd.DataFrame(pd.read_csv('/content/train.csv'))
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [103]:
df.shape

(891, 12)

# 2) DESCRIPTION OF THE ATTRIBUTES OF THE DATASET



1. Pclass: Passenger Class(1 = 1st, 2 = 2nd, 3 = 3rd)
2. survival:Survival(0 = No, 1 = Yes)
3. nam: Name
4. sex: Sex
5. age: Age
6. sibsp: Number of Siblings/Spouses Abroad
7. parch: Number of Parents/Children Abroad
8. ticket: Ticket Number
9. fare:Passenger Fare (British Pound)
10. cabin: Cabin
11. embarked: Port of Embarkation( C = Cherbourg; Q = Queenstown, S = Southampton )




























# 3) HANDELING NULL VALUES(DROPPIONG THAT ROW/COL)
For missing values:


1.   Either drop the entire row or column
2. Fill them with some appropriate value like mean,...



In [104]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Seperating out the columns which have more than 35% of the values missing in the dataset

In [105]:
drop_col= df.isnull().sum()[df.isnull().sum()>(35/100*df.shape[0])]
drop_col

Cabin    687
dtype: int64

In [106]:
drop_col.index

Index(['Cabin'], dtype='object')

In [107]:
df.drop(drop_col.index, axis=1, inplace=True)
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         2
dtype: int64

# 4) TO FILL MISSING VALUES

In [108]:
df.fillna(df.mean(), inplace=True)
df.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       2
dtype: int64

its bcz Embarked contains string values and we cannot calculate mean

In [109]:
df['Embarked'].describe()

count     889
unique      3
top         S
freq      644
Name: Embarked, dtype: object

For Embarked attribute, we fill NULL values with the most frequent value in the column 

In [110]:
df['Embarked'].fillna('S',inplace=True)

In [111]:
df.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64


# 5) TO FIND CORRELATION IN ORDER TO GROUP COLUMNS

In [112]:
df.corr()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.005007,-0.035144,0.033207,-0.057527,-0.001652,0.012658
Survived,-0.005007,1.000000,-0.338481,-0.069809,-0.035322,0.081629,0.257307
Pclass,-0.035144,-0.338481,1.000000,-0.331339,0.083081,0.018443,-0.549500
Age,0.033207,-0.069809,-0.331339,1.000000,-0.232625,-0.179191,0.091566
SibSp,-0.057527,-0.035322,0.083081,-0.232625,1.000000,0.414838,0.159651
Parch,-0.001652,0.081629,0.018443,-0.179191,0.414838,1.000000,0.216225
Fare,0.012658,0.257307,-0.549500,0.091566,0.159651,0.216225,1.000000


# 5.(1)To find Correlation of family size with survival rate 

sibsp: Number of Siblings/Spouses Aboard

parch: Number of Parents/Children Aboard

So we can make a new column family_size by combining these two columns so that the correlation is independent or 0

In [113]:
df['FamilySize']=df['SibSp']+df['Parch']
df.drop(['SibSp', 'Parch'], axis=1, inplace=True)
df.corr()

,PassengerId,Survived,Pclass,Age,Fare,FamilySize
PassengerId,1.000000,-0.005007,-0.035144,0.033207,0.012658,-0.040143
Survived,-0.005007,1.000000,-0.338481,-0.069809,0.257307,0.016639
Pclass,-0.035144,-0.338481,1.000000,-0.331339,-0.549500,0.065997
Age,0.033207,-0.069809,-0.331339,1.000000,0.091566,-0.248512
Fare,0.012658,0.257307,-0.549500,0.091566,1.000000,0.217138
FamilySize,-0.040143,0.016639,0.065997,-0.248512,0.217138,1.000000




> 
As we can observe, FamilySize in the ship doesn't have much correlation with survival rate.



# 5.(2)To find Correlation of person being Alone or not alone affects Survival rate

In [114]:
df['Alone']=[0 if df['FamilySize'][i]>0 else 1 for i in df.index]
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,Ticket,Fare,Embarked,FamilySize,Alone
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,A/5 21171,7.2500,S,1,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,PC 17599,71.2833,C,1,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,STON/O2. 3101282,7.9250,S,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,113803,53.1000,S,1,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,373450,8.0500,S,0,1


In [115]:
df.groupby(['Alone'])['Survived'].mean()

Alone
0    0.505650
1    0.303538
Name: Survived, dtype: float64

If the person is alone he/she has less chance of surviving.
The reason might be the person with his family might be belonging to rich class and might be prioritizednover other

In [116]:
df[['Alone', 'Fare']].corr()

,Alone,Fare
Alone,1.000000,-0.271832
Fare,-0.271832,1.000000


So we can see if the person was not alone, the chance the ticket price is higher is high

# 5.(3)To find Correlation of Female Passengers with Survival rate



In [117]:
df['Sex']=[0 if df['Sex'][i]=='male' else 1 for i in df.index]
df.groupby(['Sex'])['Survived'].mean()

Sex
0    0.188908
1    0.742038
Name: Survived, dtype: float64



> 
It shows that female passengers are having more chance of surviving than male ones.
> 
it shows women are prioritized over men.





# 5.(4)To find correlation of Embarked(where the train is boarded) with Survival rate

In [118]:
df.groupby(['Embarked'])['Survived'].mean()

Embarked
C    0.553571
Q    0.389610
S    0.339009
Name: Survived, dtype: float64

people who boarded from cherbourg have more survival rate.

# 6)CONCLUSION


*   Female passengers were prioritized over men.
 

*   People with high class or rich people have higher survivl rate than others. The hierarichy might have been followed
*   Passengers travelling with their family have higher survival rate

*   Passengers who borded the ship at Cherbourg, survived more in proportion then the others.


